<a href="https://colab.research.google.com/github/ggieun/h.e.after/blob/main/ROAS(%EB%A7%A4%EC%B6%9C_%EC%B0%A8%EC%9D%B4%EB%82%98%EB%8A%94_%EC%BD%94%EB%93%9C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [209]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [210]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [211]:
inventory= pd.read_csv('/content/drive/MyDrive/real_inventor.csv')
data= pd.read_csv('/content/drive/MyDrive/real_real_log.csv')

In [212]:
data.loc[data['ActionType'].isin(['view', 'basket','ad_click','share']), 'Price'] = None

In [213]:
data_purchase = data[data['ActionType'] == 'purchase'	]

In [214]:
data_adclick = data[data['ActionType'] == 'ad_click'	]

In [215]:
duplicate_rows = data_adclick[data_adclick.duplicated(keep=False)]

# 확인
print(duplicate_rows)

Empty DataFrame
Columns: [Unnamed: 0, CustomerID, ActionType, GoodsCode, Price, AdID, CPC, Adjusted_Timestamp]
Index: []


In [216]:
data_adclick_fix  = data_adclick[['CustomerID', 'ActionType', 'GoodsCode', 'Price', 'AdID', 'CPC', 'Adjusted_Timestamp']]
data_adclick_fix

,CustomerID,ActionType,GoodsCode,Price,AdID,CPC,Adjusted_Timestamp
4,74377,ad_click,1203,NaN,ad_google,175.38,2022-04-16 22:19:28.511967
23,8620,ad_click,1169,NaN,ad_meta,118.03,2022-02-08 19:54:39.511967
48,110346,ad_click,1266,NaN,ad_line,65.35,2022-05-29 17:55:12.511967
53,120663,ad_click,1050,NaN,ad_meta,45.21,2022-02-25 07:56:58.511967
62,120663,ad_click,1011,NaN,ad_line,101.81,2022-04-04 06:10:53.511967
...,...,...,...,...,...,...,...
3096826,733390,ad_click,1492,NaN,ad_meta,49.34,2022-06-06 19:44:45.393408
3096863,716526,ad_click,1472,NaN,ad_line,47.27,2022-05-31 06:01:19.395079
3096865,716526,ad_click,1472,NaN,ad_google,45.99,2022-04-22 06:54:55.395079
3096891,362992,ad_click,1382,NaN,ad_line,136.22,2022-04-30 23:01:51.397001


In [217]:
# 두데이터의 CustomerID와 GoodsCode가 같은 행조인
merged_df = pd.merge(data_purchase, data_adclick_fix, on=['CustomerID', 'GoodsCode'])
merged_df

,Unnamed: 0,CustomerID,ActionType_x,GoodsCode,Price_x,AdID_x,CPC_x,Adjusted_Timestamp_x,ActionType_y,Price_y,AdID_y,CPC_y,Adjusted_Timestamp_y
0,29,8620,purchase,1169,16754.795191,NaN,NaN,2022-03-28 04:14:44.511967,ad_click,NaN,ad_meta,118.03,2022-02-08 19:54:39.511967
1,64,110346,purchase,1266,7798.505361,NaN,NaN,2022-06-21 20:29:24.511967,ad_click,NaN,ad_line,65.35,2022-05-29 17:55:12.511967
2,69,120663,purchase,1050,11363.330139,NaN,NaN,2022-03-17 19:54:28.511967,ad_click,NaN,ad_meta,45.21,2022-02-25 07:56:58.511967
3,78,120663,purchase,1011,32937.515525,NaN,NaN,2022-05-01 13:33:59.511967,ad_click,NaN,ad_line,101.81,2022-04-04 06:10:53.511967
4,135,47804,purchase,1236,62309.187714,NaN,NaN,2022-04-13 01:53:17.511967,ad_click,NaN,ad_meta,538.95,2022-03-04 20:29:51.511967
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75137,129056,527339,purchase,1462,13015.261218,NaN,NaN,2022-05-12 09:20:33.385110,ad_click,NaN,ad_meta,69.28,2022-05-15 02:40:43.385110
75138,129162,826579,purchase,1000,12578.785587,NaN,NaN,2022-05-13 15:13:00.391379,ad_click,NaN,ad_line,120.47,2022-04-10 14:33:20.391379
75139,129182,616218,purchase,1199,9178.855085,NaN,NaN,2022-04-08 04:46:31.392190,ad_click,NaN,ad_meta,83.35,2022-05-03 07:20:49.392190
75140,129204,733390,purchase,1492,15021.277049,NaN,NaN,2022-06-17 01:17:09.393408,ad_click,NaN,ad_meta,49.34,2022-06-06 19:44:45.393408


In [218]:
merged_df['Price_x'] = merged_df['Price_x'].astype(int)

In [219]:
merged_df['Adjusted_Timestamp_x'] = pd.to_datetime(merged_df['Adjusted_Timestamp_x'])
merged_df['Adjusted_Timestamp_y'] = pd.to_datetime(merged_df['Adjusted_Timestamp_y'])

In [220]:
merged_df['DateDiff'] = (merged_df['Adjusted_Timestamp_x'] - merged_df['Adjusted_Timestamp_y']).dt.days

In [221]:
# 광고로 들어오고 7일을 초과하고 구매한 경우 제거 (구매가 광고보다 먼저일 경우도 제거)
merged_df = merged_df[merged_df['DateDiff'] < 8]
merged_df = merged_df[merged_df['DateDiff'] > 0]
merged_df

,Unnamed: 0,CustomerID,ActionType_x,GoodsCode,Price_x,AdID_x,CPC_x,Adjusted_Timestamp_x,ActionType_y,Price_y,AdID_y,CPC_y,Adjusted_Timestamp_y,DateDiff
8,188,234209,purchase,1260,20144,NaN,NaN,2022-08-26 22:03:06.511967,ad_click,NaN,ad_line,138.40,2022-08-21 07:35:10.511967,5
12,243,83371,purchase,1421,10972,NaN,NaN,2022-06-19 13:45:06.511967,ad_click,NaN,ad_google,84.46,2022-06-12 09:43:16.511967,7
25,712,241764,purchase,1336,82696,NaN,NaN,2022-03-29 18:03:03.511967,ad_click,NaN,ad_meta,205.52,2022-03-26 23:32:03.511967,2
34,1190,71512,purchase,1435,10435,NaN,NaN,2022-04-30 19:21:26.511967,ad_click,NaN,ad_google,52.07,2022-04-25 03:33:38.511967,5
51,1897,165130,purchase,1171,104650,NaN,NaN,2022-03-01 06:38:41.511967,ad_click,NaN,ad_line,835.66,2022-02-23 22:40:43.511967,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75040,125471,179405,purchase,1206,15068,NaN,NaN,2022-05-14 19:13:31.165856,ad_click,NaN,ad_google,33.18,2022-05-08 12:21:42.165856,6
75080,126601,129661,purchase,1367,42850,NaN,NaN,2022-04-27 13:24:08.235262,ad_click,NaN,ad_meta,313.14,2022-04-21 13:54:32.235262,5
75085,126883,391315,purchase,1248,6303,NaN,NaN,2022-04-18 08:25:32.252319,ad_click,NaN,ad_meta,51.79,2022-04-16 22:10:06.252319,1
75086,126887,391315,purchase,1248,6303,NaN,NaN,2022-04-24 01:50:17.252319,ad_click,NaN,ad_meta,51.79,2022-04-16 22:10:06.252319,7


In [222]:
merged_df['Price_x'].sum()

252982320

In [223]:
merged_df2 = merged_df[['CustomerID' ,'ActionType_y', 'GoodsCode',  'Price_x', 'AdID_y','CPC_y','Adjusted_Timestamp_y']]
merged_df2

,CustomerID,ActionType_y,GoodsCode,Price_x,AdID_y,CPC_y,Adjusted_Timestamp_y
8,234209,ad_click,1260,20144,ad_line,138.40,2022-08-21 07:35:10.511967
12,83371,ad_click,1421,10972,ad_google,84.46,2022-06-12 09:43:16.511967
25,241764,ad_click,1336,82696,ad_meta,205.52,2022-03-26 23:32:03.511967
34,71512,ad_click,1435,10435,ad_google,52.07,2022-04-25 03:33:38.511967
51,165130,ad_click,1171,104650,ad_line,835.66,2022-02-23 22:40:43.511967
...,...,...,...,...,...,...,...
75040,179405,ad_click,1206,15068,ad_google,33.18,2022-05-08 12:21:42.165856
75080,129661,ad_click,1367,42850,ad_meta,313.14,2022-04-21 13:54:32.235262
75085,391315,ad_click,1248,6303,ad_meta,51.79,2022-04-16 22:10:06.252319
75086,391315,ad_click,1248,6303,ad_meta,51.79,2022-04-16 22:10:06.252319


In [224]:
merged_df2.rename(columns={'ActionType_y': 'ActionType',
                   'Price_x': 'Price',
                   'AdID_y': 'AdID',
                   'CPC_y': 'CPC',
                   'Adjusted_Timestamp_y': 'Adjusted_Timestamp'
                   }, inplace=True)

merged_df2

<ipython-input-224-75f743b9d6f2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df2.rename(columns={'ActionType_y': 'ActionType',


,CustomerID,ActionType,GoodsCode,Price,AdID,CPC,Adjusted_Timestamp
8,234209,ad_click,1260,20144,ad_line,138.40,2022-08-21 07:35:10.511967
12,83371,ad_click,1421,10972,ad_google,84.46,2022-06-12 09:43:16.511967
25,241764,ad_click,1336,82696,ad_meta,205.52,2022-03-26 23:32:03.511967
34,71512,ad_click,1435,10435,ad_google,52.07,2022-04-25 03:33:38.511967
51,165130,ad_click,1171,104650,ad_line,835.66,2022-02-23 22:40:43.511967
...,...,...,...,...,...,...,...
75040,179405,ad_click,1206,15068,ad_google,33.18,2022-05-08 12:21:42.165856
75080,129661,ad_click,1367,42850,ad_meta,313.14,2022-04-21 13:54:32.235262
75085,391315,ad_click,1248,6303,ad_meta,51.79,2022-04-16 22:10:06.252319
75086,391315,ad_click,1248,6303,ad_meta,51.79,2022-04-16 22:10:06.252319


In [225]:
#merged_df2['Price'] = merged_df2['Price'].astype(int)
#merged_df2['Price'].sum()

In [226]:
concat_data = pd.concat([merged_df2, data_adclick_fix], axis=0)
concat_data.drop_duplicates(inplace=True)

In [227]:
concat_data['Adjusted_Timestamp'] = pd.to_datetime(concat_data['Adjusted_Timestamp'])

In [236]:
duplicate_rows3 = concat_data[concat_data.duplicated(keep=False)]

# 확인
duplicate_rows3.count()

CustomerID            0
ActionType            0
GoodsCode             0
Price                 0
AdID                  0
CPC                   0
Adjusted_Timestamp    0
dtype: int64

In [229]:
concat_data

,CustomerID,ActionType,GoodsCode,Price,AdID,CPC,Adjusted_Timestamp
8,234209,ad_click,1260,20144.0,ad_line,138.40,2022-08-21 07:35:10.511967
12,83371,ad_click,1421,10972.0,ad_google,84.46,2022-06-12 09:43:16.511967
25,241764,ad_click,1336,82696.0,ad_meta,205.52,2022-03-26 23:32:03.511967
34,71512,ad_click,1435,10435.0,ad_google,52.07,2022-04-25 03:33:38.511967
51,165130,ad_click,1171,104650.0,ad_line,835.66,2022-02-23 22:40:43.511967
...,...,...,...,...,...,...,...
3096826,733390,ad_click,1492,NaN,ad_meta,49.34,2022-06-06 19:44:45.393408
3096863,716526,ad_click,1472,NaN,ad_line,47.27,2022-05-31 06:01:19.395079
3096865,716526,ad_click,1472,NaN,ad_google,45.99,2022-04-22 06:54:55.395079
3096891,362992,ad_click,1382,NaN,ad_line,136.22,2022-04-30 23:01:51.397001


In [230]:
concat_data_filled = concat_data.fillna(0)
concat_data_filled

,CustomerID,ActionType,GoodsCode,Price,AdID,CPC,Adjusted_Timestamp
8,234209,ad_click,1260,20144.0,ad_line,138.40,2022-08-21 07:35:10.511967
12,83371,ad_click,1421,10972.0,ad_google,84.46,2022-06-12 09:43:16.511967
25,241764,ad_click,1336,82696.0,ad_meta,205.52,2022-03-26 23:32:03.511967
34,71512,ad_click,1435,10435.0,ad_google,52.07,2022-04-25 03:33:38.511967
51,165130,ad_click,1171,104650.0,ad_line,835.66,2022-02-23 22:40:43.511967
...,...,...,...,...,...,...,...
3096826,733390,ad_click,1492,0.0,ad_meta,49.34,2022-06-06 19:44:45.393408
3096863,716526,ad_click,1472,0.0,ad_line,47.27,2022-05-31 06:01:19.395079
3096865,716526,ad_click,1472,0.0,ad_google,45.99,2022-04-22 06:54:55.395079
3096891,362992,ad_click,1382,0.0,ad_line,136.22,2022-04-30 23:01:51.397001


In [231]:
 #중복값 삭제를 위한 데이터 프레임 생성
 #기존 데이터프레임의 복사본 만들기
new_df = merged_df2.copy()

new_df['Price'] = 0
new_df

,CustomerID,ActionType,GoodsCode,Price,AdID,CPC,Adjusted_Timestamp
8,234209,ad_click,1260,0,ad_line,138.40,2022-08-21 07:35:10.511967
12,83371,ad_click,1421,0,ad_google,84.46,2022-06-12 09:43:16.511967
25,241764,ad_click,1336,0,ad_meta,205.52,2022-03-26 23:32:03.511967
34,71512,ad_click,1435,0,ad_google,52.07,2022-04-25 03:33:38.511967
51,165130,ad_click,1171,0,ad_line,835.66,2022-02-23 22:40:43.511967
...,...,...,...,...,...,...,...
75040,179405,ad_click,1206,0,ad_google,33.18,2022-05-08 12:21:42.165856
75080,129661,ad_click,1367,0,ad_meta,313.14,2022-04-21 13:54:32.235262
75085,391315,ad_click,1248,0,ad_meta,51.79,2022-04-16 22:10:06.252319
75086,391315,ad_click,1248,0,ad_meta,51.79,2022-04-16 22:10:06.252319


In [232]:
concat_data_filled['Price'] = concat_data_filled['Price'].astype(int)
#new_df['Price'] = new_df['Price'].astype(int)
#concat_data_filled

In [233]:
# `object` 타입의 열을 `datetime64[ns]` 타입으로 변환
concat_data_filled['Adjusted_Timestamp'] = pd.to_datetime(concat_data_filled['Adjusted_Timestamp'])
new_df['Adjusted_Timestamp'] = pd.to_datetime(new_df['Adjusted_Timestamp'])

In [234]:
# A와 B를 합치고 중복 행 제거
AB_combined = pd.concat([concat_data_filled, new_df])
AB_combined

,CustomerID,ActionType,GoodsCode,Price,AdID,CPC,Adjusted_Timestamp
8,234209,ad_click,1260,20144,ad_line,138.40,2022-08-21 07:35:10.511967
12,83371,ad_click,1421,10972,ad_google,84.46,2022-06-12 09:43:16.511967
25,241764,ad_click,1336,82696,ad_meta,205.52,2022-03-26 23:32:03.511967
34,71512,ad_click,1435,10435,ad_google,52.07,2022-04-25 03:33:38.511967
51,165130,ad_click,1171,104650,ad_line,835.66,2022-02-23 22:40:43.511967
...,...,...,...,...,...,...,...
75040,179405,ad_click,1206,0,ad_google,33.18,2022-05-08 12:21:42.165856
75080,129661,ad_click,1367,0,ad_meta,313.14,2022-04-21 13:54:32.235262
75085,391315,ad_click,1248,0,ad_meta,51.79,2022-04-16 22:10:06.252319
75086,391315,ad_click,1248,0,ad_meta,51.79,2022-04-16 22:10:06.252319


In [237]:
AB_unique = AB_combined.drop_duplicates(keep= False)
AB_unique

,CustomerID,ActionType,GoodsCode,Price,AdID,CPC,Adjusted_Timestamp
8,234209,ad_click,1260,20144,ad_line,138.40,2022-08-21 07:35:10.511967
12,83371,ad_click,1421,10972,ad_google,84.46,2022-06-12 09:43:16.511967
25,241764,ad_click,1336,82696,ad_meta,205.52,2022-03-26 23:32:03.511967
34,71512,ad_click,1435,10435,ad_google,52.07,2022-04-25 03:33:38.511967
51,165130,ad_click,1171,104650,ad_line,835.66,2022-02-23 22:40:43.511967
...,...,...,...,...,...,...,...
3096826,733390,ad_click,1492,0,ad_meta,49.34,2022-06-06 19:44:45.393408
3096863,716526,ad_click,1472,0,ad_line,47.27,2022-05-31 06:01:19.395079
3096865,716526,ad_click,1472,0,ad_google,45.99,2022-04-22 06:54:55.395079
3096891,362992,ad_click,1382,0,ad_line,136.22,2022-04-30 23:01:51.397001


In [238]:
AB_unique['Price'].sum()

252115979

# **ROAS**

In [239]:
ROAS = AB_unique.groupby('AdID').agg({'GoodsCode' : 'count','CPC' : 'sum', 'Price': 'sum' })
ROAS['ROAS'] = ROAS['Price']/ROAS['CPC']*100
ROAS

,GoodsCode,CPC,Price,ROAS
AdID,,,,
ad_google,58025,11578262.55,86659598,748.468068
ad_line,56679,10893127.25,81821687,751.131288
ad_meta,57324,10846902.70,83634694,771.046780


# **CVR**

In [240]:
# 'Price' 열이 0이 아닌 값만 카운트하기
count_nonzero = AB_unique[AB_unique['Price'] != 0].groupby('AdID').agg({'ActionType': 'count', 'Price': 'count'})
total_count = AB_unique.groupby('AdID').agg({'ActionType': 'count'})

# 'ActionType' 열이 유효한 값의 개수를 합산하기
count_nonzero['ActionType'] = total_count['ActionType'] - count_nonzero['ActionType']
count_nonzero

,ActionType,Price
AdID,,
ad_google,55357,2668
ad_line,54082,2597
ad_meta,54732,2592


In [241]:
count_nonzero.rename(columns={'ActionType': 'ad_click',
                              'Price': 'purchase'}, inplace=True)

In [242]:
CVR = count_nonzero['purchase'] / count_nonzero['ad_click']*100
CVR

AdID
ad_google    4.819625
ad_line      4.801967
ad_meta      4.735804
dtype: float64